In [1]:
import polars as pl
import torch

# Analysis of Chip-Seq data

in the narrowPeak format the peak score correspond to the column_7
the *set* file correspond to the peak calling on the merge annotation to have the same peak positions

we load the files

In [2]:
files_H3K4me3 = [
    "../data/chipseq/H3K4me3_wt_R1_peaks.narrowPeak",
    "../data/chipseq/H3K4me3_wt_R2_peaks.narrowPeak",
    "../data/chipseq/H3K4me3_wt_R3_peaks.narrowPeak",
]
narrowpeak_schema = {
    'chrom': pl.Utf8,
    'start': pl.Int64,
    'end': pl.Int64,
    'name': pl.Utf8,
    'score': pl.Int64,
    'strand': pl.Utf8,
    'signal_value': pl.Float64,
    'pvalue': pl.Float64,
    'qvalue': pl.Float64,
    'peak': pl.Int64  # Relative peak position
}

data_H3K4me3 = [pl.read_csv(file, separator="\t", has_header=False, schema=narrowpeak_schema) for file in files_H3K4me3]
torch.tensor(data_H3K4me3[0]['signal_value'].to_list())

tensor([3.0552, 2.1930, 1.2766,  ..., 1.8311, 1.4504, 1.3172])

Then we only keep the common peak between the 3 narrowPeak line by finding the peak that overlapp between the 3 files

In [3]:
# Function to find overlapping peaks
def find_overlapping_peaks(df1, df2, min_overlap, sufix=2):
    """Find peaks that overlap by at least min_overlap bases"""
    
    # Cross join to compare all peak pairs within same chromosome
    overlaps = df1.join(df2, on='chrom', how='inner', suffix=f"_{sufix}")
    
    # Calculate overlap
    overlaps = overlaps.with_columns([
        # Overlap start is the maximum of both starts
        pl.max_horizontal(['start', f"start_{sufix}"]).alias('overlap_start'),
        # Overlap end is the minimum of both ends  
        pl.min_horizontal(['end', f"end_{sufix}"]).alias('overlap_end')
    ]).with_columns([
        # Calculate overlap length
        (pl.col('overlap_end') - pl.col('overlap_start')).alias('overlap_length')
    ]).filter(
        # Keep only actual overlaps
        pl.col('overlap_length') >= min_overlap
    )
    
    return overlaps

In [4]:
min_overlap=1

# Start with first dataset
merged = data_H3K4me3[0]
    
   # Iteratively merge with others
for i, next_df in enumerate(data_H3K4me3[1:], 1):
    print(f"Merging with sample {files_H3K4me3[i]}...")
    
    # Find overlaps
    merged = find_overlapping_peaks(merged, next_df, min_overlap, 1+i)
merged

Merging with sample ../data/chipseq/H3K4me3_wt_R2_peaks.narrowPeak...
Merging with sample ../data/chipseq/H3K4me3_wt_R3_peaks.narrowPeak...


shape: (14_207, 31)
┌───────┬──────────┬──────────┬────────────────┬───┬────────────────┬──────────┬──────────┬────────┐
│ chrom ┆ start    ┆ end      ┆ name           ┆ … ┆ signal_value_3 ┆ pvalue_3 ┆ qvalue_3 ┆ peak_3 │
│ ---   ┆ ---      ┆ ---      ┆ ---            ┆   ┆ ---            ┆ ---      ┆ ---      ┆ ---    │
│ str   ┆ i64      ┆ i64      ┆ str            ┆   ┆ f64            ┆ f64      ┆ f64      ┆ i64    │
╞═══════╪══════════╪══════════╪════════════════╪═══╪════════════════╪══════════╪══════════╪════════╡
│ chrI  ┆ 3783     ┆ 4920     ┆ H3K4me3_wt_R1_ ┆ … ┆ 9.10734        ┆ 146.047  ┆ 143.751  ┆ 470    │
│       ┆          ┆          ┆ peak_1         ┆   ┆                ┆          ┆          ┆        │
│ chrI  ┆ 11348    ┆ 11903    ┆ H3K4me3_wt_R1_ ┆ … ┆ 4.95983        ┆ 47.411   ┆ 45.9114  ┆ 463    │
│       ┆          ┆          ┆ peak_2         ┆   ┆                ┆          ┆          ┆        │
│ chrI  ┆ 12320    ┆ 12664    ┆ H3K4me3_wt_R1_ ┆ … ┆ 2.59561        ┆ 13.7958  ┆ 12.6147  ┆ 267    │
│       ┆          ┆          ┆ peak_3         ┆   ┆                ┆          ┆          ┆        │
│ chrI  ┆ 12776    ┆ 13676    ┆ H3K4me3_wt_R1_ ┆ … ┆ 2.59561        ┆ 13.7958  ┆ 12.6147  ┆ 267    │
│       ┆          ┆          ┆ peak_4         ┆   ┆                ┆          ┆          ┆        │
│ chrI  ┆ 14902    ┆ 15112    ┆ H3K4me3_wt_R1_ ┆ … ┆ 4.83873        ┆ 54.4632  ┆ 52.9122  ┆ 2167   │
│       ┆          ┆          ┆ peak_5         ┆   ┆                ┆          ┆          ┆        │
│ …     ┆ …        ┆ …        ┆ …              ┆ … ┆ …              ┆ …        ┆ …        ┆ …      │
│ chrX  ┆ 17627403 ┆ 17629386 ┆ H3K4me3_wt_R1_ ┆ … ┆ 5.09898        ┆ 47.9801  ┆ 46.4759  ┆ 631    │
│       ┆          ┆          ┆ peak_16618     ┆   ┆                ┆          ┆          ┆        │
│ chrX  ┆ 17631735 ┆ 17633016 ┆ H3K4me3_wt_R1_ ┆ … ┆ 2.61564        ┆ 11.5443  ┆ 10.3979  ┆ 1154   │
│       ┆          ┆          ┆ peak_16622     ┆   ┆                ┆          ┆          ┆        │
│ chrX  ┆ 17711129 ┆ 17711795 ┆ H3K4me3_wt_R1_ ┆ … ┆ 4.40552        ┆ 36.1245  ┆ 34.7127  ┆ 1019   │
│       ┆          ┆          ┆ peak_16624     ┆   ┆                ┆          ┆          ┆        │
│ chrX  ┆ 17711848 ┆ 17712322 ┆ H3K4me3_wt_R1_ ┆ … ┆ 4.40552        ┆ 36.1245  ┆ 34.7127  ┆ 1019   │
│       ┆          ┆          ┆ peak_16625     ┆   ┆                ┆          ┆          ┆        │
│ chrX  ┆ 17714663 ┆ 17715318 ┆ H3K4me3_wt_R1_ ┆ … ┆ 2.16197        ┆ 6.9183   ┆ 5.8605   ┆ 148    │
│       ┆          ┆          ┆ peak_16626     ┆   ┆                ┆          ┆          ┆        │
└───────┴──────────┴──────────┴────────────────┴───┴────────────────┴──────────┴──────────┴────────┘

We only consider the *signal_value* columns

In [5]:
signal_H3K4me3 = torch.stack([torch.tensor(merged['signal_value'].to_list())] + [torch.tensor(merged[f'signal_value_{i}'].to_list()) for i in [2,3]], dim=1)
mask = (signal_H3K4me3 != 0).all(dim=1)  # True for rows with no zeros
signal_H3K4me3 = signal_H3K4me3[mask]

In [6]:
import torch
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode

# Initialize offline mode for notebooks
init_notebook_mode(connected=True)

# Convert to numpy
coords = signal_H3K4me3.numpy()

# Create interactive 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1], 
    z=coords[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=coords[:, 2],  # Color by z-coordinate
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="signal value")
    ),
    text=[f'Point {i}' for i in range(len(coords))],  # Hover text
    hovertemplate='<b>%{text}</b><br>R1: %{x:.2f}<br>R2: %{y:.2f}<br>R3: %{z:.2f}<extra></extra>'
)])

fig.update_layout(
    title='H3K4me3',
    scene=dict(
        xaxis_title='R1 Axis',
        yaxis_title='R2 Axis',
        zaxis_title='R3 Axis'
    ),
    width=800,
    height=600
)
iplot(fig)

## IDR computation for H3K4me3

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
!uv pip install ../
import torch
import midr
import importlib
importlib.reload(midr)
torch.set_default_dtype(torch.float64)
torch.set_printoptions(precision=10)
torch.set_printoptions(sci_mode=False)

Resolved 20 packages in 177ms                                        
Prepared 1 package in 1.21s                                              
Uninstalled 1 package in 15ms
Installed 1 package in 3msile:///Users/laurent/projects/phys
 ~ midr==0.1.0 (from file:///Users/laurent/projects/physbio/midrpy)


In [9]:
ecdf_H3K4me3 = midr.auxilary.ecdf(signal_H3K4me3).detach().numpy()

In [10]:
import numpy as np
print(ecdf_H3K4me3[np.any(np.isnan(ecdf_H3K4me3), axis=1)])
print(signal_H3K4me3[np.any(np.isnan(ecdf_H3K4me3), axis=1)])
print(ecdf_H3K4me3[np.any((ecdf_H3K4me3 < 0.0) | (ecdf_H3K4me3 > 1.0), axis=1)])
print(signal_H3K4me3[np.any((ecdf_H3K4me3 < 0.0) | (ecdf_H3K4me3 > 1.0), axis=1)])

[]
tensor([], size=(0, 3), dtype=torch.float32)
[]
tensor([], size=(0, 3), dtype=torch.float32)


In [11]:
# Initialize offline mode for notebooks
init_notebook_mode(connected=True)

# Convert to numpy
coords = midr.auxilary.ecdf(signal_H3K4me3).detach().numpy()

# Create interactive 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1], 
    z=coords[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=coords[:, 2],  # Color by z-coordinate
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="signal value")
    ),
    text=[f'Point {i}' for i in range(len(coords))],  # Hover text
    hovertemplate='<b>%{text}</b><br>R1: %{x:.2f}<br>R2: %{y:.2f}<br>R3: %{z:.2f}<extra></extra>'
)])

fig.update_layout(
    title='H3K4me3',
    scene=dict(
        xaxis_title='R1 Axis',
        yaxis_title='R2 Axis',
        zaxis_title='R3 Axis'
    ),
    width=800,
    height=600
)
iplot(fig)

In [12]:
!uv pip install ../
import torch
import midr
import importlib
importlib.reload(midr)
torch.set_default_dtype(torch.float64)
torch.set_printoptions(precision=10)
torch.set_printoptions(sci_mode=False)
idr_H3K4me3, fdr_H3K4me3 = midr.compute_idr(data=signal_H3K4me3, copula="archmixture", pseudo_data=True, gpu=False, verbose=True, progress=True)

Resolved 20 packages in 14ms                                         
Prepared 1 package in 1.10s                                              
Uninstalled 1 package in 9ms
Installed 1 package in 2msile:///Users/laurent/projects/phys
 ~ midr==0.1.0 (from file:///Users/laurent/projects/physbio/midrpy)
Iteration 0: Loss = 52057.59497582456, theta = tensor([ 0.7281005989, -0.2187886482, -0.2892060102, -0.2201059404,
         0.8507894078,  0.9125072332,  0.9338429742, 14.9168574467,
        27.9672435384,  9.9209588091], requires_grad=True)
Iteration 1: Loss = 47282.67409373127, theta = tensor([ 0.7281062731, -0.2187918454, -0.2892053825, -0.2201090452,
         0.8508101726,  0.9125447274,  0.9338539828, 14.9168621716,
        27.9672454274,  9.9209633198], requires_grad=True)
Iteration 2: Loss = 47282.53774397515, theta = tensor([ 0.7281062895, -0.2187918546, -0.2892053807, -0.2201090542,
         0.8508102323,  0.9125448357,  0.9338540146, 14.9168621852,
        27.9672454329,  9.92096

In [13]:
# Initialize offline mode for notebooks
init_notebook_mode(connected=True)

# Convert to numpy
coords = midr.auxilary.ecdf(signal_H3K4me3).detach().numpy()

# Create interactive 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1], 
    z=coords[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=idr_H3K4me3.detach().numpy(),  # Color by idr
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="signal value")
    ),
    text=[f'Point {i}' for i in range(len(coords))],  # Hover text
    hovertemplate='<b>%{text}</b><br>R1: %{x:.2f}<br>R2: %{y:.2f}<br>R3: %{z:.2f}<extra></extra>'
)])

fig.update_layout(
    title='H3K4me3',
    scene=dict(
        xaxis_title='R1 Axis',
        yaxis_title='R2 Axis',
        zaxis_title='R3 Axis'
    ),
    width=800,
    height=600
)
iplot(fig)

AttributeError: module 'midr' has no attribute 'auxilary'

In [ ]:
# Initialize offline mode for notebooks
init_notebook_mode(connected=True)

# Convert to numpy
coords = midr.auxilary.ecdf(signal_H3K4me3).detach().numpy()

# Create interactive 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=coords[:, 0],
    y=coords[:, 1], 
    z=coords[:, 2],
    mode='markers',
    marker=dict(
        size=2,
        color=fdr_H3K4me3.detach().numpy(),  # Color by fdr
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(title="signal value")
    ),
    text=[f'Point {i}' for i in range(len(coords))],  # Hover text
    hovertemplate='<b>%{text}</b><br>R1: %{x:.2f}<br>R2: %{y:.2f}<br>R3: %{z:.2f}<extra></extra>'
)])

fig.update_layout(
    title='H3K4me3',
    scene=dict(
        xaxis_title='R1 Axis',
        yaxis_title='R2 Axis',
        zaxis_title='R3 Axis'
    ),
    width=800,
    height=600
)
iplot(fig)